In [1]:
import hoomd
import hoomd.md
import numpy

# Accessing system configurations in python

To get some data for an example, run the Lennard-Jones tutorial script.

In [2]:
hoomd.context.initialize('--mode=cpu');
system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=5);
nl = hoomd.md.nlist.cell();
lj = hoomd.md.pair.lj(r_cut=3.0, nlist=nl);
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0);
all = hoomd.group.all();
hoomd.md.integrate.mode_standard(dt=0.001);
hoomd.md.integrate.langevin(group=all, kT=1.0, seed=987);

HOOMD-blue v2.1.5 CUDA (7.5) DOUBLE HPMC_MIXED MPI SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX 
Compiled: 03/12/2017
Copyright 2009-2016 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 125 particles
notice(2): integrate.langevin/bd is using specified gamma values


## Take snapshots

Use `take_snapshot` to access the current system configuration as a snapshot.

In [3]:
snap = system.take_snapshot(all=True)

The snapshot contains the simulation box, and particle properties. It also includes bond, angle, dihedral, and constraint topology information which will be covered in an MD specific tutorial. For a complete list of particle properties in the snapshot, see the snapshot documentation.

In [4]:
print(snap.box)

Box: Lx=10.0 Ly=10.0 Lz=10.0 xy=0.0 xz=0.0 yz=0.0 dimensions=3


In [5]:
snap.particles.position[0:10,:]

array([[-4., -4., -4.],
       [-4., -4., -2.],
       [-4., -4.,  0.],
       [-4., -4.,  2.],
       [-4., -4.,  4.],
       [-4., -2., -4.],
       [-4., -2., -2.],
       [-4., -2.,  0.],
       [-4., -2.,  2.],
       [-4., -2.,  4.]], dtype=float32)

In [6]:
snap.particles.velocity[0:10,:]

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]], dtype=float32)

The snapshot object is an independent copy of the simulation data. When you continue the simulation run, the snapshot remains unchanged.

In [7]:
hoomd.run(10e3, quiet=True)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 125
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no


In [8]:
snap.particles.position[0:10,:]

array([[-4., -4., -4.],
       [-4., -4., -2.],
       [-4., -4.,  0.],
       [-4., -4.,  2.],
       [-4., -4.,  4.],
       [-4., -2., -4.],
       [-4., -2., -2.],
       [-4., -2.,  0.],
       [-4., -2.,  2.],
       [-4., -2.,  4.]], dtype=float32)

When you call `take_snapshot` again, you will get the updated system configuration.

In [9]:
snap2 = system.take_snapshot(all=True)
snap2.particles.position[0:10,:]

array([[ 2.23450947,  4.14357567,  3.23013973],
       [ 1.9669745 , -2.60286117,  2.52477765],
       [-1.535514  ,  0.18256012, -4.43649864],
       [-0.8033995 , -1.77420795,  1.3952688 ],
       [-4.49599552, -0.79098874,  3.91530466],
       [ 1.04309428, -4.68264437,  4.53090811],
       [ 2.28033209, -1.71373463,  0.69547868],
       [ 0.29374596,  2.2471056 , -4.04393196],
       [-2.71024776,  1.25365555, -0.87069672],
       [ 2.04618239, -0.15408203,  1.57929945]], dtype=float32)

## Analyze system configurations periodically

Perform analysis tasks periodically on the system configuration using a callback analyzer. First, create the analyzer callback class that captures the system variable saved in the initialization command.

In [10]:
class avg_vel:
    def __init__(self, system):
        self.system = system;
    def __call__(self, timestep):
        snap = self.system.take_snapshot();
        avg_vel = numpy.mean(snap.particles.velocity, axis=0);
        print(timestep, ':', avg_vel);

Then, create the callback analyzer and attach it to the callback class.

In [11]:
hoomd.analyze.callback(callback=avg_vel(system), period=100);

Run the simulation and observe the output of the analysis.

In [12]:
hoomd.run(1000, quiet=True);

(10000L, ':', array([-0.11201318, -0.02595793, -0.12756611], dtype=float32))
(10100L, ':', array([-0.07834366,  0.00738541, -0.11726347], dtype=float32))
(10200L, ':', array([-0.04684083,  0.04181128, -0.05382856], dtype=float32))
(10300L, ':', array([-0.03593112,  0.01279185, -0.13130777], dtype=float32))
(10400L, ':', array([-0.01960432, -0.01791668, -0.08600815], dtype=float32))
(10500L, ':', array([-0.05530114, -0.02656797, -0.13475499], dtype=float32))
(10600L, ':', array([-0.07956874, -0.04796169, -0.02593341], dtype=float32))
(10700L, ':', array([-0.06260108, -0.12288381,  0.04341358], dtype=float32))
(10800L, ':', array([-0.07554695, -0.11343727, -0.00279859], dtype=float32))
(10900L, ':', array([-0.02769233, -0.13621315, -0.01962889], dtype=float32))
